### 유전자 알고리즘 간단 테스트

- 문제 정의 
  - 팀원(14명)의 점심 조를 짜 보자. 단, 아래 제약 조건 고려   

- 제약조건 
  - 14명을 4개 조로 나누기
  - 조별 인원은 3~4명
  - 조별로 시니어(선임,수석매니저) 1명 이상 포함 (조별로 시니어 비중 균일하게)
  - 조별로 남자 1명 이상 포함 (조별로 성별 비중 균일하게)

- 유전자 알고리즘 적용 
  - 제약조건을 패털티로 가산하는 최적화 함수 작성
  - 유전자 알고리즘에서는 교배, 변이 등을 발생시켜 최적화 함수의 패널티를 최소화하는 최적해를 찾는다

- 유전자 알고리즘 참조 사이트 
  - https://pygad.readthedocs.io/en/latest/
  - https://data-newbie.tistory.com/m/685

In [2]:
!pip install pygad   # 라이브러러 설치 

In [ ]:
import pygad 

from collections import Counter

import numpy as np
import pandas as pd

### 학습용 데이터 정의

- 팀원 : 14명 
- 팀원별 시니어 여부, 남자 여부를 이진 벡터로 구성 

In [ ]:
desired_output = 0.0000001   # 적합도 함수 최적값 : 패널티 값이 0

senior = [1,1,1,1,1,1,0,0,0,0,0,0,0,0] # 선임이상 = 1 
male = [1,1,1,1,1,1,1,0,1,0,0,0,1,0]   # 성별남자 = 1
name = ['KYJ', 'YHH', 'KHD', 'LIH', 'KMG', 'LDH', 'LDB', 'LYJ', 'LWR', 'PJH', 'JGE', 'SHJ', 'PJT', 'LDK'] 

In [ ]:
def fitness_func(solution, solution_idx):
  """  
      - 패널티 구조식 정의 : 4가지 조건별 =
      - (추가과제) 항목별 패널티 가중치 점수 조합을, 그리드 서치로 찾아봐도 재밌을 듯 
  """
  penalty = 0		            # 패널티 점수 초기화  
  freq = Counter(solution)  # GA에서 만든 14명의 조 리스트(solution)의 건수 집계, dict 저장

  penalty += (len(freq)-4)*100    # (조건#1) 조의 갯수가 4개가 아니면 패널티 
 
  for value in freq.values():     # (조건#2) 조별 인원수 3,4명 아닌 경우 패널티
    if value not in (3,4):
      penalty += 10

  # (조건#3,4) 조별 선임이상, 남성 비율이 평균 보다 크거나 작으면 패널티
  df = pd.DataFrame({'part':solution, 'senior':senior, 'male':male})

  senior_avg = df['senior'].mean()
  penalty += np.abs( df.groupby(['part'])['senior'].mean().values - senior_avg).sum() * 11

  male_avg = df['male'].mean()
  penalty += np.abs( df.groupby(['part'])['male'].mean().values - male_avg).sum() * 5
    
  fitness = 1 / np.abs(penalty - desired_output)  #  0에 가까울수록 4가지 조건을 만족하는 최적해
  return fitness
	
ga_instance = pygad.GA(num_generations=50,
                       num_parents_mating=4,
                       fitness_func=fitness_func,
                       sol_per_pop=8,
                       num_genes=14,      # 입력값 갯수 
                       init_range_low=0,  # 값 하한 
                       init_range_high=4, # 값 상한 
                       parent_selection_type='sss',
                       keep_parents=1,
                       crossover_type="single_point",
                       mutation_type="random",
                       mutation_percent_genes=10,
                       gene_type=int)    # 정수형 벡터 생성. gene_type = float가 기본 값

#### 4가지 조건을 모두 만족하는 최적해가 나오면 결과를 출력하고, 반복을 중단한다. 

In [ ]:
while True:
  ga_instance.run()
  solution, solution_fitness, solution_idx = ga_instance.best_solution()
  
  res = pd.DataFrame({'name':name, 'part':solution, 'senior':senior, 'male':male})  
  if res.groupby('part')['senior'].mean().min() > 0 and res.groupby('part')['male'].mean().min() > 0:
    break

print("Solution Fitness Values : {}".format(solution_fitness))
print("\nSenior Pct by Part : ", res.groupby('part')['senior'].mean()*100)
print("\nMale Pct by Part : ", res.groupby('part')['male'].mean()*100)
print("\nResult : ")
res.sort_values(by=['part', 'senior', 'male'])

Solution Fitness Values : 0.19399538482577655

Senior Pct by Part :  part
0    50.000000
1    33.333333
2    33.333333
3    50.000000
Name: senior, dtype: float64

Male Pct by Part :  part
0    75.000000
1    66.666667
2    66.666667
3    50.000000
Name: male, dtype: float64

Result : 


,name,part,senior,male
11,SHJ,0,0,0
8,LWR,0,0,1
1,YHH,0,1,1
4,KMG,0,1,1
13,LDK,1,0,0
6,LDB,1,0,1
5,LDH,1,1,1
9,PJH,2,0,0
12,PJT,2,0,1
3,LIH,2,1,1
